In [1]:
import pandas as pd

# Load the datasets
# Replace these file paths with your actual file paths
driver_dataset_path = 'Driver_Dataset_with_Ages.csv'
race_details_path = 'Race_Details_with_Ages.csv'
output_path = 'Race_Details_with_Ages_Complete.csv'

# Read the CSV files
print("Loading datasets...")
driver_df = pd.read_csv(driver_dataset_path)
race_details_df = pd.read_csv(race_details_path)

# Display initial information
print(f"Driver dataset shape: {driver_df.shape}")
print(f"Race details dataset shape: {race_details_df.shape}")

# Check for the correct column names
print("\nDriver dataset columns:", driver_df.columns.tolist())
print("Race details columns:", race_details_df.columns.tolist())

# Ensure the driverId column names match (sometimes there might be case differences)
# Standardize column names if needed
if 'driverid' in driver_df.columns:
    driver_df.rename(columns={'driverid': 'driverId'}, inplace=True)

# Create a subset of driver data with just the necessary columns
driver_subset = driver_df[['driverId', 'name']].copy()

# Remove any duplicate driver entries (if any)
driver_subset = driver_subset.drop_duplicates(subset='driverId')

print(f"\nUnique drivers in driver dataset: {len(driver_subset)}")
print(f"Unique drivers in race details: {race_details_df['driverId'].nunique()}")

# Merge the driver names with the race details
print("\nMerging driver names with race details...")
race_details_with_names = race_details_df.merge(
    driver_subset,
    on='driverId',
    how='left'
)

# Reorder columns to have driver name next to driverId
cols = race_details_with_names.columns.tolist()
# Find positions of driverId and name
driver_id_idx = cols.index('driverId')
name_idx = cols.index('name')

# Remove 'name' from its current position
cols.pop(name_idx)
# Insert 'name' right after 'driverId'
cols.insert(driver_id_idx + 1, 'name')

# Reorder the dataframe
race_details_with_names = race_details_with_names[cols]

# Check for any missing driver names
missing_names = race_details_with_names[race_details_with_names['name'].isna()]
if not missing_names.empty:
    print(f"\nWarning: {len(missing_names)} records have missing driver names")
    print("Driver IDs without names:", missing_names['driverId'].unique().tolist())
else:
    print("\nSuccess: All driver names have been mapped!")

# Display sample of the merged data
print("\nSample of merged data (first 10 rows):")
print(race_details_with_names[['raceId', 'year', 'driverId', 'name', 'age_during_race']].head(10))

# Save the updated dataset
race_details_with_names.to_csv(output_path, index=False)
print(f"\nUpdated dataset saved to: {output_path}")

# Display statistics
print("\n=== Statistics ===")
print(f"Total race records: {len(race_details_with_names)}")
print(f"Drivers with age data: {race_details_with_names['age_during_race'].notna().sum()}")
print(f"Drivers with names: {race_details_with_names['name'].notna().sum()}")

# Optional: Create a summary of driver ages during races
print("\n=== Age Analysis ===")
age_stats = race_details_with_names[race_details_with_names['age_during_race'].notna()].groupby('name')['age_during_race'].agg([
    ('min_age', 'min'),
    ('max_age', 'max'),
    ('avg_age', 'mean'),
    ('race_count', 'count')
]).round(1)

print("\nTop 10 drivers by number of races with age data:")
print(age_stats.sort_values('race_count', ascending=False).head(10))

Loading datasets...
Driver dataset shape: (912, 22)
Race details dataset shape: (7909, 18)

Driver dataset columns: ['driverid', 'name', 'abbreviation', 'countryOfBirthCountryId', 'bestStartingGridPosition', 'bestRaceResult', 'totalChampionshipWins', 'totalRaceEntries', 'totalRaceStarts', 'totalRaceWins', 'totalRaceLaps', 'totalPodiums', 'totalPoints', 'totalChampionshipPoints', 'totalPolePositions', 'totalFastestLaps', 'totalGrandSlams', 'date_of_birth', 'date_of_death', 'current_age', 'age_f1_entry', 'age_retirement']
Race details columns: ['raceId', 'year', 'round', 'positionDisplayOrder', 'driverId', 'constructorId', 'laps', 'points', 'polePosition', 'qualificationPositionNumber', 'positionsGained', 'pitStops', 'fastestLap', 'grandSlam', 'temperature', 'precipitation', 'windspeed', 'age_during_race']

Unique drivers in driver dataset: 912
Unique drivers in race details: 113

Merging driver names with race details...

Success: All driver names have been mapped!

Sample of merged dat